In [2]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from numpy import argmax, array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2


In [3]:
train = pd.read_csv("kdd_train.csv")
test = pd.read_csv("kdd_test.csv")

print(test.shape,train.shape)
#print(test['duration'])

(22544, 42) (125973, 42)


In [4]:
for i in ['protocol_type', 'service', "flag", "labels"]:
    values_train = array(train[i])
    values_test = array(test[i])
    label_encoder = LabelEncoder()
    integer_encoded_train = label_encoder.fit_transform(values_train)
    integer_encoded_test = label_encoder.fit_transform(values_test)
    train[i] = integer_encoded_train
    test[i] = integer_encoded_test

X_train = train.iloc[:, :41]
y_train = train.iloc[:, 41]

X_test = test.iloc[:, :41]
y_test = test.iloc[:, 41]

#print(X_train.head(),y_train.head())
print(X_test.head(),y_test.head())
#print(list(y_train))

duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0         5              1       51     9       2429        475     0   
1         0              2       12     9         45        134     0   
2         0              2       12     9         45         80     0   
3      1979              2       41     9        145        105     0   
4     14462              1       41     4          1          0     0   

   wrong_fragment  urgent  hot  ...  dst_host_count  dst_host_srv_count  \
0               0       0    0  ...             180                  57   
1               0       0    0  ...             194                 155   
2               0       0    0  ...             255                 255   
3               0       0    0  ...             255                   2   
4               0       0    0  ...             255                   2   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.11                    0.02   
1      

In [6]:

rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=20)
# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
rfe_X_train = pd.DataFrame()
rfe_X_test = pd.DataFrame()
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    col_name_train = train[train.columns[i]]
    col_name_test = test[test.columns[i]]
    if rfe.support_[i]:
        rfe_X_train = pd.concat([rfe_X_train, col_name_train], axis = 1)
        rfe_X_test = pd.concat([rfe_X_test, col_name_test], axis = 1)
      
print (rfe_X_train.shape, rfe_X_test.head())
print(col_name_train)


Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected False, Rank: 2.000
Column: 6, Selected False, Rank: 5.000
Column: 7, Selected True, Rank: 1.000
Column: 8, Selected False, Rank: 14.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 3.000
Column: 11, Selected False, Rank: 8.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 12.000
Column: 14, Selected False, Rank: 18.000
Column: 15, Selected False, Rank: 15.000
Column: 16, Selected False, Rank: 10.000
Column: 17, Selected False, Rank: 9.000
Column: 18, Selected False, Rank: 19.000
Column: 19, Selected False, Rank: 21.000
Column: 20, Selected False, Rank: 22.000
Column: 21, Selected False, Rank: 20.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 7.000
Column: 24, Selected True, Rank: 1.000
Column: 25,

In [7]:
#ELM
input_size = rfe_X_train.shape[1]
hidden_size = 20

input_weights = np.random.uniform(-5,5,size=(input_size,hidden_size))
#input_weights = np.random.normal(size=[input_size, hidden_size])
biases = np.random.normal(size=[hidden_size])

print(input_weights,biases)


[[ 4.21070320e+00  3.87351083e+00 -4.68262300e+00  1.40342188e+00
  -3.44133012e+00  8.91082409e-01 -4.32393514e+00 -4.48365064e+00
   3.05354569e+00  1.38371230e+00  3.80441794e+00 -1.92678114e+00
  -4.37559915e+00 -2.02974517e+00  1.38452213e+00 -8.55795008e-01
  -3.54894330e-01 -2.13663949e-01  1.95140072e+00 -3.11413889e+00]
 [-1.77651291e+00  3.47453315e-01  1.06027848e+00  2.53767634e+00
   1.03378345e+00  4.02577455e+00  3.90714781e-03  4.31443714e-01
  -4.56144709e+00 -3.04294127e+00  3.73009575e+00 -3.45677876e+00
  -3.04526800e+00 -4.01196529e+00 -2.74948053e+00  4.34272850e+00
   2.74503445e+00 -2.32210490e+00 -1.70259470e-01  9.69827531e-01]
 [ 2.80494908e+00  1.67017110e+00 -1.63569681e-01  4.85713577e+00
   6.79146616e-01  7.88807072e-01 -1.46035505e+00 -1.24402082e+00
   3.04609525e-01 -3.71978676e+00 -4.45192632e+00  4.22574711e+00
  -3.58279775e+00 -1.11058918e+00 -4.40608792e+00 -1.25745555e-01
   6.62248105e-01  2.77445669e+00 -2.42468955e+00  4.37390386e+00]
 [ 2.50

In [8]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + biases
    H = sigmoid(G)
    return H

output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train)), y_train)

<ipython-input-8-1ed99a0c591d>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [9]:

def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

prediction = predict(rfe_X_test)
print (prediction, y_test)

[11.0046301  10.87291129 11.10246734 ...  9.19423828 12.01924327
  9.50544566] 0        16
1        16
2        16
3        16
4        20
         ..
22539    25
22540    16
22541    14
22542    20
22543    16
Name: labels, Length: 22544, dtype: int64
<ipython-input-8-1ed99a0c591d>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [12]:

correct = 0
total = rfe_X_test.shape[0]
sum1 = 0

for i in range(total):
    predicted = prediction[i]
    actual = y_test[i]
    #print(prediction[i], predicted, y[i], actual)
    sum1+= ((actual - predicted)**2)
    #print(sum1)

mse = sum1/total

print('Accuracy for ', hidden_size, ' hidden nodes: ', mse)

Accuracy for  20  hidden nodes:  47.35212458537501
